# Reference

 - http://pen.envr.tsukuba.ac.jp/~torarimon/?GDAL_python
 - https://gis-oer.github.io/gitbook/book/materials/python/07/07.html

# Setting path

In [1]:
# original file 
orgdir = 'orgdata/ALAV2A200132900-OORIRFU-D071P3-20091027-001/'

# Clip

In [2]:
import glob
fs = glob.glob(orgdir + 'IMG*')
fs

['orgdata/ALAV2A200132900-OORIRFU-D071P3-20091027-001\\IMG-01-ALAV2A200132900-OORIRFU-D071P3-20091027-001.tif',
 'orgdata/ALAV2A200132900-OORIRFU-D071P3-20091027-001\\IMG-02-ALAV2A200132900-OORIRFU-D071P3-20091027-001.tif',
 'orgdata/ALAV2A200132900-OORIRFU-D071P3-20091027-001\\IMG-03-ALAV2A200132900-OORIRFU-D071P3-20091027-001.tif',
 'orgdata/ALAV2A200132900-OORIRFU-D071P3-20091027-001\\IMG-04-ALAV2A200132900-OORIRFU-D071P3-20091027-001.tif']

In [3]:
from osgeo import gdal
import os

clip = 'clip.geojson'
for f in fs:
    fout = os.path.dirname(f) + '/clip-' + os.path.basename(f)
    d = gdal.Warp(fout, f, dstSRS="EPSG:6676", xRes=10.0, yRes=10.0, cutlineDSName=clip, cropToCutline=True, dstNodata="0")
    d = None

# Overlay

In [4]:
import glob
fs = glob.glob(orgdir + 'clip*')
fs.sort()
fs

['orgdata/ALAV2A200132900-OORIRFU-D071P3-20091027-001\\clip-IMG-01-ALAV2A200132900-OORIRFU-D071P3-20091027-001.tif',
 'orgdata/ALAV2A200132900-OORIRFU-D071P3-20091027-001\\clip-IMG-02-ALAV2A200132900-OORIRFU-D071P3-20091027-001.tif',
 'orgdata/ALAV2A200132900-OORIRFU-D071P3-20091027-001\\clip-IMG-03-ALAV2A200132900-OORIRFU-D071P3-20091027-001.tif',
 'orgdata/ALAV2A200132900-OORIRFU-D071P3-20091027-001\\clip-IMG-04-ALAV2A200132900-OORIRFU-D071P3-20091027-001.tif']

In [5]:
# make new file from original file profile
ds = gdal.Open(fs[0], gdal.GA_ReadOnly)
arr = ds.GetRasterBand(1).ReadAsArray()
h, w = arr.shape
dst_ds = gdal.GetDriverByName('GTiff').Create("tenryurivmouth.tif", w, h, 4, gdal.GDT_Byte)
d = dst_ds.SetGeoTransform(ds.GetGeoTransform())
d = dst_ds.SetProjection(ds.GetProjection())

# overlay band data
for i, f in enumerate(fs):
    ds = gdal.Open(f, gdal.GA_ReadOnly)
    x = ds.ReadAsArray()
    d = dst_ds.GetRasterBand(i+1).WriteArray(x)

# post
ds.FlushCache()    
dst_ds = None